In [1]:
import awkward as ak
import numpy as np
import pandas as pd
from tabulate import tabulate

from apps.prod4a_merge_study import (BestCut, EventSelection, PFOSelection,
                                     ShowerMergeQuantities)
from python.analysis import Master, Plots

nEvents = 500
events = Master.Data("work/ROOTFiles/Prod4a_6GeV_BeamSim_00.root", True)#, _nEvents=nEvents)
cuts = "csv/cuts/analysedCuts-6GeV.csv"
#cuts = "test/new-csv/analysedCuts.csv"

start_showers = EventSelection(events)
#start_showers = PFOSelection(events, start_showers)
mask = np.logical_or(events.recoParticles.cnnScore > 0.64, np.logical_or(*start_showers)) # select PFOs which pass CNN selection or are start showers
events.Filter([mask])
start_showers = [start_showers[i][mask] for i in range(2)]
to_merge = np.logical_not(np.logical_or(*start_showers))

quantities = ShowerMergeQuantities(events, to_merge, cuts)
quantities.bestCut = "purity"

n_merge = -1

print("calculating score...")
def SortByStartingShower(data):
    data = [ak.unflatten(data[i], 1, -1) for i in range(2)]
    return ak.concatenate(data, -1)

def ClosestQuantity(q : ak.Array, mask : ak.Array):
    masked_q = ak.where(mask, q, 9999999)
    q_to_merge = ak.argmin(masked_q, -1, keepdims=True)
    return ak.where(ak.min(masked_q, -1, keepdims=True) == 9999999, -1, q_to_merge)

#* retrieve quantities and find which start shower is closest to each PFO for each variable
quantities.Evaluate(events, start_showers)
print("evaluated quantities")
mask = SortByStartingShower(quantities.mask) # PFOs we want to merge after cut based selection is done
alpha = ClosestQuantity(SortByStartingShower(quantities.alpha), mask) # can use this to determine which starting shower the PFO is closest to in angle
x = ClosestQuantity(SortByStartingShower(quantities.delta_x), mask) # can use this to determine which starting shower the PFO is closest to in space
phi = ClosestQuantity(SortByStartingShower(quantities.delta_phi), mask) # can use this to determine which starting shower the PFO direction is most aligned to
print(mask)

#* figure out which is the common start shower between all variables
# if min phi, alpha and x are all the same then merge to that shower
# if two are the same, merge to the most common shower
# if none agree (shouldn't be possible)
#! should replace this with calculating the mode of the scores
scores_raw = ak.sum(ak.concatenate([phi, x, alpha], -1), -1)
scores = ak.where(scores_raw == 1, 0, scores_raw) # [1, 0, 0]
scores = ak.where(scores == 2, 1, scores) # [1, 1, 0]
scores = ak.where(scores == 3, 1, scores) # [1, 1, 1]

/home/sb16165/Documents/pi0-analysis/analysis/python/analysis/Master.py:50: UserWarning: Couldn't apply filters to filters.
  warnings.warn(f"Couldn't apply filters to {var}.")
/home/sb16165/Documents/pi0-analysis/analysis/python/analysis/Master.py:50: UserWarning: Couldn't apply filters to filename.
  warnings.warn(f"Couldn't apply filters to {var}.")
/home/sb16165/Documents/pi0-analysis/analysis/python/analysis/Master.py:50: UserWarning: Couldn't apply filters to _TrueParticleData__pi0_MC.
  warnings.warn(f"Couldn't apply filters to {var}.")


'BeamMCFilter' executed in 14.6280s
number of dalitz decays: 222
'ApplyBeamFilter' executed in 9.2544s


\begin{tabular}{lllll}
\hline
 event selection                   & type        & number of events & percentage of events removed & percentage of 
events remaining \\
 no selection                      & -           & 100222           & -                            &               
\\
 beam -\ensuremath{>} pi0 + X                   & truth       & 27341            & 72.71956257109217            & 
100                            \\
 pi+ beam                          & backtracked & 19949            & 27.036319081233312           & 
72.96368091876668              \\
 diphoton decay                    & truth       & 19727            & 1.1128377362273798           & 
72.15171354376211              \\
 beam particle                     & reco        & 10017            & 49.22187864348355            & 
36.637284664057646             \\
 nPFP \ensuremath{>} 1                          & reco        & 10017            & 0.0                          & 
36.637284664057646             \\
 at least 1 true particle          & backtracked & 10017            & 0.0                          & 
36.637284664057646             \\
 both true photons are backtracked & backtracked & 4356             & 56.51392632524708            & 
15.932116601441052             \\
\hline
\end{tabular}

False

[714, 549, 497, 433, 441, 545, 402, 790, ... 528, 930, 527, 511, 496, 675, 778, 953]

...

cutting on: delta_phi

cutting on: d

cutting on: t

cutting on: p

'Evaluate' executed in 275.7681s
evaluated quantities
[[[False, False], [False, False], [False, ... [False, False], [False, False]]]


In [5]:
def ShowerMergingPFOPerformance(start_showers, to_merge, scores, quantities):
    # false negative - showers we should have merged but didn't
    # false positive - showers we merged but shouldn't have
    # true positive - showers we should have merged and did
    # true negative - showers we should have merged but didn't
    # mismatch - of the showers merged, which were assigned to the wrong start shower

    all_showers = np.logical_or(*start_showers)
    s_num = events.trueParticlesBT.number[all_showers]
    tm_num = events.trueParticlesBT.number[to_merge]

    signals = [s_num[:, i] == tm_num for i in range(2)]
    signal_all = np.logical_or(*signals)[quantities.null] # showers we should have merged
    background = np.logical_not(signal_all) # showers we shouldn't have merged

    merged = scores != -3 # PFOs actually merged
    not_merged = np.logical_not(merged)

    print(ak.count(merged))
    print(ak.count(signal_all))

    tp = np.logical_and(merged, signal_all) # true positive, signal pfos merged
    tn = np.logical_not(np.logical_or(merged, signal_all)) # true negative, background not merged

    xor = np.logical_xor(merged, signal_all)

    fp = np.logical_and(xor, signal_all == False) # false positive, background PFOs merged
    fn = np.logical_and(xor, signal_all == True) # false negative, signal PFOs not merged

    nSignal = ak.count(signal_all[signal_all])
    nBackground = ak.count(background[background])
    nMerged = ak.count(merged[merged])
    nUnmerged = ak.count(not_merged[not_merged])

    print(f"number of signal PFOs before cutting: {nSignal}")
    print(f"number of background PFOs before cutting: {nBackground}")
    confusion = [
        ["performance metric", "number of PFOs", "percentage (signal/background)", "percentage (merged, unmerged)"],
        ["signal PFOs merged (true positives)", ak.count(tp[tp]), 100 * ak.count(tp[tp])/nSignal, 100 * ak.count(tp[tp])/nMerged],
        ["background PFOs merged (false positive)", ak.count(fp[fp]), 100 * ak.count(fp[fp])/nBackground, 100 * ak.count(fp[fp])/nMerged],
        ["signal PFOs not merged (false negative)", ak.count(fn[fn]), 100 * ak.count(fn[fn])/nSignal, 100 * ak.count(fn[fn])/nUnmerged],
        ["background PFOs not merged (true negatives)", ak.count(tn[tn]), 100 * ak.count(tn[tn])/nBackground, 100 * ak.count(tn[tn])/nUnmerged]
        ]

    print(f"scores: {ak.count(scores)}")
    print(f"to_merge: {ak.count(to_merge[quantities.null])}")

    signal_num = events.trueParticlesBT.number[to_merge][quantities.null][signal_all]
    target_num = events.trueParticlesBT.number[to_merge][quantities.null][tp]
    actual_num = ak.where(scores == 0, s_num[:, 0], scores)
    actual_num = ak.where(actual_num == 1, s_num[:, 1], actual_num)
    actual_num = actual_num[tp]

    actual_num = actual_num[ak.num(actual_num) > 0]
    target_num = target_num[ak.num(target_num) > 0]

    mismatch = ak.ravel(actual_num == target_num)
    print(f"mismatch (%): {100 * ak.count(mismatch[mismatch == False]) / ak.count(mismatch)}")

    print(tabulate(confusion, floatfmt=".2f", tablefmt="fancy_grid"))

    mask = quantities.mask
    cut_signal = signal_all[np.logical_or(*mask)]
    print(ak.count(ak.ravel(cut_signal[cut_signal])))
    globals().update(locals()) #! debugging only!

ShowerMergingPFOPerformance(start_showers, to_merge, scores, quantities)

996607
996607
number of signal PFOs before cutting: 13116
number of background PFOs before cutting: 983491
scores: 996607
to_merge: 996607
mismatch (%): 21.865889212827987
╒═════════════════════════════════════════════╤════════════════╤════════════════════════════════╤═══════════════════════════════╕
│ performance metric                          │ number of PFOs │ percentage (signal/background) │ percentage (merged, unmerged) │
├─────────────────────────────────────────────┼────────────────┼────────────────────────────────┼───────────────────────────────┤
│ signal PFOs merged (true positives)         │ 1715           │ 13.075632814882587             │ 55.44778532169415             │
├─────────────────────────────────────────────┼────────────────┼────────────────────────────────┼───────────────────────────────┤
│ background PFOs merged (false positive)     │ 1378           │ 0.14011312762394368            │ 44.55221467830585             │
├─────────────────────────────────────────────┼─

In [4]:
def ShowerMergingEventPerformance(start_showers, to_merge, scores, quantities):
    all_showers = np.logical_or(*start_showers)
    s_num = events.trueParticlesBT.number[all_showers]
    tm_num = events.trueParticlesBT.number[to_merge]

    signal = np.logical_or(*[tm_num == s_num[:, i] for i in range(2)])[quantities.null] # showers we should have merged
    background = np.logical_not(signal) # showers we shouldn't have merged

    merged = scores != -3 # PFOs actually merged
    not_merged = np.logical_not(merged)

    nMerged = ak.count(merged[merged], -1)
    nSignal = ak.count(signal[signal], -1)
    nBackground = ak.count(background[background], -1)

    tp = np.logical_and(merged, signal) # true positive
    nTp = ak.count(tp[tp], -1)

    xor = np.logical_xor(merged, signal)

    fp = np.logical_and(xor, signal == False) # false positive
    nFp = ak.count(fp[fp], -1)
    t = np.logical_and(nFp > 0, nTp > 0)

    signal_only = np.logical_and(nFp == 0, nTp > 0)
    background_only = np.logical_and(nFp > 0, nTp == 0)

    print(nMerged)
    print(nSignal)
    print(nBackground)
    print(f"number of events: {ak.count(events.eventNum)}")
    print(f"number of events with PFOs to merge: {ak.count(nSignal[nSignal > 0])}")
    print(f"number of events where we merge: {ak.count(nMerged[nMerged > 0])}")
    print(f"number of events where we merge signal: {ak.count(nTp[nTp > 0])}")
    print(f"number of events where we merge background: {ak.count(nFp[nFp > 0])}")
    print(f"number of events where we merge signal and background {ak.count(t[t])}")
    print(f"number of events where we merge only signal {ak.count(signal_only[signal_only])}")
    print(f"number of events where we merge only background {ak.count(background_only[background_only])}")

ShowerMergingEventPerformance(start_showers, to_merge, scores, quantities)

[0, 0, 1, 0, 0, 0, 2, 1, 0, 0, 1, 1, 0, 0, ... 0, 3, 0, 0, 0, 1, 0, 0, 0, 0, 1, 3, 2]
[2, 1, 0, 4, 0, 0, 0, 0, 2, 0, 2, 7, 0, ... 1, 2, 0, 2, 1, 6, 3, 20, 0, 1, 5, 7, 18]
[300, 220, 159, 161, 165, 224, 137, 270, ... 228, 306, 155, 182, 185, 288, 271, 325]
number of events: 4356
number of events with PFOs to merge: 3334
number of events where we merge: 1802
number of events where we merge signal: 1089
number of events where we merge background: 1003
number of events where we merge signal and background 290
number of events where we merge only signal 799
number of events where we merge only background 713
